In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch

#setting the device 
device = torch.device('mps')
print(f'all imports successfull and the device name is {device}')

all imports successfull and the device name is mps
